# Clase 2: Pensando en Tablas (El Modelo Relacional)

Bienvenidos a la segunda clase. Hoy dejaremos de ver los datos como "listas" sueltas y empezaremos a pensar en **Sistemas Relacionales**.

## Objetivo de Aprendizaje
- Entender qué es una Base de Datos Relacional (RDBMS).
- Diferenciar entre Tablas, Filas (Tuplas) y Columnas (Atributos).
- Diseñar un esquema básico conectando entidades con **Claves Primarias (PK)** y **Claves Foráneas (FK)**.
- Crear tablas usando SQL.

---

## Paso 1: Configuración de Kaggle (Repaso)
Ejecuta estas celdas para tener acceso al dataset, igual que en la clase anterior.

In [ ]:
from google.colab import files
import os

if not os.path.exists("/root/.kaggle/kaggle.json"):
    print("Por favor, sube tu archivo kaggle.json:")
    uploaded = files.upload()
    
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("Las credenciales de Kaggle ya están configuradas.")

In [ ]:
# Descargamos el dataset si no existe
if not os.path.exists("baseball_data"):
    !kaggle datasets download -d open-source-sports/baseball-databank
    !unzip -q baseball-databank.zip -d baseball_data
    print("Dataset descargado y descomprimido.")
else:
    print("El dataset ya está listo.")

---

## Paso 2: El Modelo Relacional

En deportes, todo está conectado. Un **Jugador** pertenece a un **Equipo**, juega **Partidos** y genera **Eventos** (Goles, Hits, Faltas).

### Conceptos Clave
1.  **Tabla (Entidad):** Una colección de objetos del mismo tipo (ej. `Jugadores`).
2.  **ID / Primary Key (PK):** Un identificador único para cada fila (ej. `player_id`). Nunca se repite.
3.  **Foreign Key (FK):** Un campo que apunta al ID de otra tabla (ej. `team_id` en la tabla de Jugadores). Así es como relacionamos los datos.

## Paso 3: Práctica - Diseñando un Esquema SQL

Vamos a diseñar una pequeña base de datos para registrar eventos de partidos. Usaremos **SQLite** (incluido en Python) para crear las tablas.

In [ ]:
import sqlite3
import pandas as pd

# Conectamos a una base de datos temporal en memoria
# Esto significa que si cerramos el cuaderno, los datos se pierden (ideal para aprender)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
print("Conexión a base de datos SQLite establecida.")

### Creando la tabla EQUIPOS
Esta tabla tendrá el ID del equipo y su nombre.

In [ ]:
create_teams_query = """
CREATE TABLE equipos (
    team_id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    ciudad TEXT
);
"""
cursor.execute(create_teams_query)
print("Tabla 'equipos' creada.")

### Creando la tabla JUGADORES
Aquí usaremos una **Foreign Key** (`team_id`) para decir a qué equipo pertenece el jugador.

In [ ]:
create_players_query = """
CREATE TABLE jugadores (
    player_id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    posicion TEXT,
    numero_camiseta INTEGER,
    team_id INTEGER, 
    FOREIGN KEY (team_id) REFERENCES equipos (team_id)
);
"""
cursor.execute(create_players_query)
print("Tabla 'jugadores' creada.")

### Insertando Datos de Prueba
Vamos a poblar nuestras tablas con datos ficticios para ver cómo funciona.

In [ ]:
# Insertar Equipos
cursor.execute("INSERT INTO equipos (team_id, nombre, ciudad) VALUES (1, 'Tigres', 'Cancún')")
cursor.execute("INSERT INTO equipos (team_id, nombre, ciudad) VALUES (2, 'Leones', 'Mérida')")

# Insertar Jugadores (Noten como usamos team_id 1 o 2)
cursor.execute("INSERT INTO jugadores VALUES (101, 'Ivan R.', 'Pitcher', 45, 1)")
cursor.execute("INSERT INTO jugadores VALUES (102, 'Carlos M.', 'Catcher', 12, 1)")
cursor.execute("INSERT INTO jugadores VALUES (201, 'Jose P.', 'Outfield', 99, 2)")

conn.commit()
print("Datos insertados correctamente.")

## Paso 4: Verificando las Relaciones (JOIN)
Ahora haremos una consulta SQL simple para ver el nombre del jugador junto con el nombre de su equipo. ¡Esta es la magia de las bases de datos relacionales!

In [ ]:
query = """
SELECT 
    j.nombre as Jugador, 
    j.posicion as Posicion, 
    e.nombre as Equipo
FROM jugadores j
JOIN equipos e ON j.team_id = e.team_id
"""

# Usamos pandas para ver el resultado bonito
df_resultado = pd.read_sql_query(query, conn)
df_resultado

### Ejercicio Final
1.  Intenta insertar un nuevo equipo.
2.  Intenta insertar un jugador asignado a ese nuevo equipo.
3.  Vuelve a ejecutar la consulta JOIN para ver si aparece.